In [ ]:
import pandas as pd
import commonHelper
from dataDownloader import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_stock import DB_Stock
from db_nyse import DB_NYSE
from commonHelper import EDateType, EFinancialStatementType
from datetime import datetime
import yfinance as yf
from portfolio import Portfolio
import numpy as np
from scipy.stats import norm
from dateutil.parser import parse
import itertools
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta

# pd.set_option('display.max_rows', None)  # 모든 행 출
# pd.set_option('display.max_columns', None)  # 모든 열 출력
# pd.set_option('display.expand_frame_repr', False)  # 긴 데이터 프레임 줄바꿈 없이 출력
# pd.set_option('display.max_colwidth', None)  # 열 하나에 들어가는 텍스트의 최대 길이를 제한 없이 설정
# pd.set_option('display.width', None)         # 줄바꿈 없이 전체 데이터 프레임을 한 줄로 출력

# 필요한 지표
DB_FinancialStatement.show_rank_by_fs()



2022-01-01 00:00:00 이전 상장된 기업 추출
2022 이전 상장 티커 수 : 20
🔍 검사할 분기 범위: ['2024-Q2', '2024-Q3', '2024-Q4', '2025-Q1']
✅ 범위 내 모든 분기를 가진 Symbol 수: 11
✅ 교집합 Quarters: ['2024-Q2', '2024-Q3', '2024-Q4', '2025-Q1']
⚠️ 교집합이 줄어든 원인 Symbol(가장 적은 분기 보유): ['AAP']
📊 이들 Symbol의 보유 분기 수: 4


,2025-Q1
0,ABG
1,AA
2,AAOI
3,ABL
4,AAP
5,ABEV
6,AAL
7,A
8,AAON
9,ABBV


<built-in method now of type object at 0x1015ea9b8>
